## Combinatorial Optimization

### Solving the The Traveling Salesman Problem - TSP

**Authors:** Guilherme C. Cadori; Rafael A. de Oliveira

**Date:** 20/09/2023

#### 1) Loading Instances

In [1]:
# Importing libraries
# !pip install gurobipy
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random as rdm
import gurobipy as gp
import time
import requests
import pandas as pd
import itertools
import numba as nb
import scipy
from io import StringIO


In [2]:
# Define a list of URLs to fetch data from
urls = [
    'https://raw.githubusercontent.com/guilhermecadori/TSPData/main/bays29.tsp',
    'https://raw.githubusercontent.com/guilhermecadori/TSPData/main/att48.tsp',
    'https://raw.githubusercontent.com/guilhermecadori/TSPData/main/berlin52.tsp',
    'https://raw.githubusercontent.com/guilhermecadori/TSPData/main/kroA100.tsp',
    'https://raw.githubusercontent.com/guilhermecadori/TSPData/main/ch130.tsp'
]

# Create an empty dictionary to store datasets
datasets = {}

print('Loading instances:')

# Loop through the URLs and fetch data for each one
for url in urls:
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.text

        # Split the data into lines
        lines = data.split('\n')

        # Get the dimension from the 4th row (assuming it follows the format "DIMENSION: <value>")
        dimension_line = lines[3].strip()
        dimension = int(dimension_line.split(":")[1].strip())

        # Initialize an empty list to store the data
        data_list = []

        # Start reading data from the 7th line until "EOF" is encountered
        for line in lines[6:]:
            if line.strip() == 'EOF':
                break
            else:
                columns = line.split()
                data_list.append(columns)

        # Create a Pandas DataFrame from the data list
        df = pd.DataFrame(data_list, columns=['nodeID', 'coord1', 'coord3'])
        df = df.astype({'nodeID': 'float64', 'coord1': 'float64', 'coord3': 'float64'})

        # Create a dataset name that includes the dimension
        dataset_name = f'Instance{dimension}'

        # Store the dataset in the dictionary with the name as the key
        datasets[dataset_name] = df

        loadedInstanceCount = len(datasets.keys())

        print(f'Loaded instance {loadedInstanceCount}')

    else:
        print(f'Failed to retrieve data from {url}. Status code: {response.status_code}')

# Counting the all the loaded instances
totalInstanceCount = len(datasets.keys())

print(f'\nSuccessfully loaded {totalInstanceCount} instances.')

# Checking created datasets
print('\nThe following instances were created:')

for dataset in datasets.keys():
    print(dataset)


Loading instances:
Loaded instance 1
Loaded instance 2
Loaded instance 3
Loaded instance 4
Loaded instance 5

Successfully loaded 5 instances.

The following instances were created:
Instance29
Instance48
Instance52
Instance100
Instance130


In [3]:
# Creating distance matrices
distance_matrices = {}

for dataset_name, dataset_df in datasets.items():
    distance_matrices[dataset_name] = scipy.spatial.distance.cdist(dataset_df, dataset_df)

print('\nCalculations completed. Check if all expected matrices were produced\n')

# Now, 'distance_matrices' contains the distance matrices for each dataset
# Checking matrices' names
print('\nThe following distance matrices were created:')

for dataset in distance_matrices.keys():
    print(dataset)



Calculations completed. Check if all expected matrices were produced


The following distance matrices were created:
Instance29
Instance48
Instance52
Instance100
Instance130


In [4]:
# You can access a specific distance matrix by its dataset name, e.g., distance_matrices["Instance48"]
# Check if matrices were created correctly
# pd.DataFrame(distance_matrices['Instance29'])
# pd.DataFrame(distance_matrices['Instance48'])
# pd.DataFrame(distance_matrices['Instance52'])
# pd.DataFrame(distance_matrices['Instance100'])
# pd.DataFrame(distance_matrices['Instance130'])


In [5]:
#### Instância 1
# Creating pair-wise distance dictionaries
# Number of nodes (assuming it's a square matrix)
num_nodes1 = len(distance_matrices['Instance29'])

# Initialize an empty dictionary to store the pair-wise distances
pairwise_distances1 = {}

# Iterate through each node
for i in range(num_nodes1):
    for j in range(num_nodes1):
        # Skip calculating distance between the same node
        if i == j:
            continue
        # Create a key for the pair (i, j) and store the distance
        pairwise_distances1[(i, j)] = distance_matrices['Instance29'][i, j]

# Now, pairwise_distances will contain the distances between each pair of nodes
# pairwise_distances1


#### Instância 2
num_nodes2 = len(distance_matrices['Instance48'])

# Initialize an empty dictionary to store the pair-wise distances
pairwise_distances2 = {}

# Iterate through each node
for i in range(num_nodes2):
    for j in range(num_nodes2):
        # Skip calculating distance between the same node
        if i == j:
            continue
        # Create a key for the pair (i, j) and store the distance
        pairwise_distances2[(i, j)] = distance_matrices['Instance48'][i, j]


#### Instância 3
num_nodes3 = len(distance_matrices['Instance52'])

# Initialize an empty dictionary to store the pair-wise distances
pairwise_distances3 = {}

# Iterate through each node
for i in range(num_nodes3):
    for j in range(num_nodes3):
        # Skip calculating distance between the same node
        if i == j:
            continue
        # Create a key for the pair (i, j) and store the distance
        pairwise_distances3[(i, j)] = distance_matrices['Instance52'][i, j]


#### Instância 4
num_nodes4 = len(distance_matrices['Instance100'])

# Initialize an empty dictionary to store the pair-wise distances
pairwise_distances4 = {}

# Iterate through each node
for i in range(num_nodes4):
    for j in range(num_nodes4):
        # Skip calculating distance between the same node
        if i == j:
            continue
        # Create a key for the pair (i, j) and store the distance
        pairwise_distances4[(i, j)] = distance_matrices['Instance100'][i, j]


#### Instância 5
num_nodes5 = len(distance_matrices['Instance130'])

# Initialize an empty dictionary to store the pair-wise distances
pairwise_distances5 = {}

# Iterate through each node
for i in range(num_nodes5):
    for j in range(num_nodes5):
        # Skip calculating distance between the same node
        if i == j:
            continue
        # Create a key for the pair (i, j) and store the distance
        pairwise_distances5[(i, j)] = distance_matrices['Instance130'][i, j]


#### 2) Solving Approach: Pure Heuristic

In [6]:
def tour_length(instance, distance_matrices, tour):
    total_distance = 0
    for i in range(len(tour)):
        total_distance += distance_matrices[instance][tour[i]][tour[(i+1) % len(tour)]]
    return total_distance

In [7]:
def calculate_edge_changes(instance, distance_matrices, tour, i, j):
    n = len(tour)
    prev_i = (i - 1 + n) % n
    next_i = (i + 1) % n
    prev_j = (j - 1 + n) % n
    next_j = (j + 1) % n

    if abs(i-j) == 1 or abs(i-j) == n - 1:
        old_length = (
            distance_matrices[instance][tour[prev_i]][tour[i]] +
            distance_matrices[instance][tour[i]][tour[next_i]] +
            distance_matrices[instance][tour[prev_j]][tour[j]] +
            distance_matrices[instance][tour[j]][tour[next_j]] -
            distance_matrices[instance][tour[i]][tour[j]]
        )
        new_length = (
            distance_matrices[instance][tour[prev_i]][tour[next_i]] +
            distance_matrices[instance][tour[prev_j]][tour[next_j]] +
            distance_matrices[instance][tour[i]][tour[j]]
        )
    else:
        old_length = (
            distance_matrices[instance][tour[prev_i]][tour[i]] +
            distance_matrices[instance][tour[i]][tour[next_i]] +
            distance_matrices[instance][tour[prev_j]][tour[j]] +
            distance_matrices[instance][tour[j]][tour[next_j]]
        )
        new_length = (
            distance_matrices[instance][tour[prev_j]][tour[i]] +
            distance_matrices[instance][tour[i]][tour[next_j]] +
            distance_matrices[instance][tour[prev_i]][tour[j]] +
            distance_matrices[instance][tour[j]][tour[next_i]]
        )

    return new_length - old_length

In [8]:
def lin_kernighan(instance, distance_matrices, tour):
    n = len(tour)
    improvement = True

    while improvement:
        improvement = False

        for i, j in itertools.combinations(range(n), 2):
            delta_length = calculate_edge_changes(instance, distance_matrices, tour, i, j)

            if delta_length < 0:
                tour[i], tour[j] = tour[j], tour[i]
                improvement = True
                break

    return tour

In [9]:
for instance in datasets.keys():
    initial_tour = rdm.sample(list(range(len(datasets[instance]))), len(datasets[instance]))
    initial_length = tour_length(instance, distance_matrices, initial_tour)
    improved_tour = lin_kernighan(instance, distance_matrices, initial_tour)
    improved_length = tour_length(instance, distance_matrices, improved_tour)
    print(instance,' - ', 'Starting Solution: ', round(initial_length, 2),' | ', 'Final Solution: ', round(improved_length, 2), sep='')

Instance29 - Starting Solution: 26815.83 | Final Solution: 10105.22
Instance48 - Starting Solution: 144310.43 | Final Solution: 39947.3
Instance52 - Starting Solution: 26996.45 | Final Solution: 10716.73
Instance100 - Starting Solution: 162213.34 | Final Solution: 42103.31
Instance130 - Starting Solution: 48611.76 | Final Solution: 12772.75


bays29 - 1610 \
att48 - 10628 \
berlin52 - 7542 \
kroa100 - 21282 \
ch130 - 6110 

In [24]:
import sys
sys.version

'3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]'

In [10]:
# pip install lk_heuristic

In [11]:
# from lk_heuristic.utils.solver_funcs import solve

In [12]:
# solve()

#### 3) Solving Approach: Pure Mathematical Model

##### 3.1) Instância 1

In [13]:
# # Instance range
# inst1 = range(len(distance_matrices['Instance29']))
# print(inst1)


In [14]:
# # Instance size
# num_nodes = distance_matrices['Instance29'].shape[0]
# node_ids1 = list(range(num_nodes))
# print(node_ids1)


In [15]:
# pairwise_distances1

In [16]:
# Instance range
inst1 = range(len(distance_matrices['Instance29']))
# print(inst1)

# Instance size
num_nodes = distance_matrices['Instance29'].shape[0]
node_ids1 = list(range(num_nodes))
# print(nodesInst1)

# Criando o modelo
m1 = gp.Model()

# Criando variávies: nesse caso todos o nós são adjacente entre si
variables = m1.addVars(pairwise_distances1.keys(),
                     obj=pairwise_distances1,
                     vtype=gp.GRB.BINARY,
                     name='x')

# Direções simétricas: copiando objeto
for u, v in pairwise_distances1.keys():
    variables[v, u] = variables[u, v] # mesmo arco porém na direção oposta

# Criando restrições
# Em cada nó pode haver somente dois arcos selecionados - um nó de chegada e outro nó de saída
cons = m1.addConstrs(variables.sum(node,'*') == 2 for node in inst1)

#### Registrando tempo de início
start_time = time.time()

# Adicionando restrições de sub-rotas de Dantzig de forma iterativa
# As restrições serão inseridas quando sub-rotas forem criadas
def subtourelim(model, where):
    # Importando biblioteca auxiliar
    from itertools import combinations

    if where == gp.GRB.Callback.MIPSOL:
        # Criando uma lista de arcos selecionados
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)

        # Encontrando a menor subrota criada dentro da atual lista de arcos selecionados
        tour = subtour(selected)
        if len(tour) < len(node_ids1):
            # Adiciona restrição de eliminação de sub-rotas para o par de cidade na subrota
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Dada uma sublista de arcos encontre a menor subrota
def subtour(edges):
    unvisited = node_ids1[:]
    cycle = node_ids1[:] # Proxy - será substituída
    while unvisited:  # 'True' se a lista não estiver vazia
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nova menor subrota
    return cycle

# Resolvendo o modelo
m1._vars = variables
m1.Params.lazyConstraints = 1
m1.optimize(subtourelim)

#### Registrando tempo de término
end_time = time.time()

# Solução
vals = m1.getAttr('x', variables)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

# Rota a ser traçada
rota1 = subtour(selected)

#### Calculando tempo decorrido
elapsed_time_DT1 = round(end_time - start_time, 4)

print('________________________________________________________________')
#### Retornando solução
print('\nInstância 1 - Função Objetivo: R$', round(m1.objVal, 2))

#### Retornando tempo decorrido
print("\nElapsed time:", elapsed_time_DT1, "seconds")
print('\nRota:', rota1)
print('________________________________________________________________')


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-07
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 29 rows, 812 columns and 812 nonzeros
Model fingerprint: 0xc0a0dc88
Variable types: 0 continuous, 812 integer (812 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.01s
Presolved: 29 rows, 812 columns, 812 nonzeros
Variable types: 0 continuous, 812 integer (812 binary)

Root relaxation: objective 8.944332e+03, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

##### 3.2) Instância 2

In [17]:
# Instance range
inst1 = range(len(distance_matrices['Instance48']))
# print(inst1)

# Instance size
num_nodes = distance_matrices['Instance48'].shape[0]
node_ids1 = list(range(num_nodes))
# print(nodesInst1)

# Criando o modelo
m1 = gp.Model()

# Criando variávies: nesse caso todos o nós são adjacente entre si
variables = m1.addVars(pairwise_distances2.keys(),
                     obj=pairwise_distances2,
                     vtype=gp.GRB.BINARY,
                     name='x')

# Direções simétricas: copiando objeto
for u, v in pairwise_distances2.keys():
    variables[v, u] = variables[u, v] # mesmo arco porém na direção oposta

# Criando restrições
# Em cada nó pode haver somente dois arcos selecionados - um nó de chegada e outro nó de saída
cons = m1.addConstrs(variables.sum(node,'*') == 2 for node in inst1)

#### Registrando tempo de início
start_time = time.time()

# Adicionando restrições de sub-rotas de Dantzig de forma iterativa
# As restrições serão inseridas quando sub-rotas forem criadas
def subtourelim(model, where):
    # Importando biblioteca auxiliar
    from itertools import combinations

    if where == gp.GRB.Callback.MIPSOL:
        # Criando uma lista de arcos selecionados
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)

        # Encontrando a menor subrota criada dentro da atual lista de arcos selecionados
        tour = subtour(selected)
        if len(tour) < len(node_ids1):
            # Adiciona restrição de eliminação de sub-rotas para o par de cidade na subrota
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Dada uma sublista de arcos encontre a menor subrota
def subtour(edges):
    unvisited = node_ids1[:]
    cycle = node_ids1[:] # Proxy - será substituída
    while unvisited:  # 'True' se a lista não estiver vazia
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nova menor subrota
    return cycle

# Resolvendo o modelo
m1._vars = variables
m1.Params.lazyConstraints = 1
m1.optimize(subtourelim)

#### Registrando tempo de término
end_time = time.time()

# Solução
vals = m1.getAttr('x', variables)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

# Rota a ser traçada
rota1 = subtour(selected)

#### Calculando tempo decorrido
elapsed_time_DT1 = round(end_time - start_time, 4)

print('________________________________________________________________')
#### Retornando solução
print('\nInstância 2 - Função Objetivo: R$', round(m1.objVal, 2))

#### Retornando tempo decorrido
print("\nElapsed time:", elapsed_time_DT1, "seconds")
print('\nRota:', rota1)
print('________________________________________________________________')


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 2256 columns and 2256 nonzeros
Model fingerprint: 0x7aeef8e3
Variable types: 0 continuous, 2256 integer (2256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 48 rows, 2256 columns, 2256 nonzeros
Variable types: 0 continuous, 2256 integer (2256 binary)

Root relaxation: objective 3.169020e+04, 77 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 31690.2032    0   14          - 31690.

##### 3.3) Instância 3

In [18]:
# Instance range
inst1 = range(len(distance_matrices['Instance52']))
# print(inst1)

# Instance size
num_nodes = distance_matrices['Instance52'].shape[0]
node_ids1 = list(range(num_nodes))
# print(nodesInst1)

# Criando o modelo
m1 = gp.Model()

# Criando variávies: nesse caso todos o nós são adjacente entre si
variables = m1.addVars(pairwise_distances3.keys(),
                     obj=pairwise_distances3,
                     vtype=gp.GRB.BINARY,
                     name='x')

# Direções simétricas: copiando objeto
for u, v in pairwise_distances3.keys():
    variables[v, u] = variables[u, v] # mesmo arco porém na direção oposta

# Criando restrições
# Em cada nó pode haver somente dois arcos selecionados - um nó de chegada e outro nó de saída
cons = m1.addConstrs(variables.sum(node,'*') == 2 for node in inst1)

#### Registrando tempo de início
start_time = time.time()

# Adicionando restrições de sub-rotas de Dantzig de forma iterativa
# As restrições serão inseridas quando sub-rotas forem criadas
def subtourelim(model, where):
    # Importando biblioteca auxiliar
    from itertools import combinations

    if where == gp.GRB.Callback.MIPSOL:
        # Criando uma lista de arcos selecionados
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)

        # Encontrando a menor subrota criada dentro da atual lista de arcos selecionados
        tour = subtour(selected)
        if len(tour) < len(node_ids1):
            # Adiciona restrição de eliminação de sub-rotas para o par de cidade na subrota
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Dada uma sublista de arcos encontre a menor subrota
def subtour(edges):
    unvisited = node_ids1[:]
    cycle = node_ids1[:] # Proxy - será substituída
    while unvisited:  # 'True' se a lista não estiver vazia
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nova menor subrota
    return cycle

# Resolvendo o modelo
m1._vars = variables
m1.Params.lazyConstraints = 1
m1.optimize(subtourelim)

#### Registrando tempo de término
end_time = time.time()

# Solução
vals = m1.getAttr('x', variables)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

# Rota a ser traçada
rota1 = subtour(selected)

#### Calculando tempo decorrido
elapsed_time_DT1 = round(end_time - start_time, 4)

print('________________________________________________________________')
#### Retornando solução
print('\nInstância 3 - Função Objetivo: R$', round(m1.objVal, 2))

#### Retornando tempo decorrido
print("\nElapsed time:", elapsed_time_DT1, "seconds")
print('\nRota:', rota1)
print('________________________________________________________________')


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 52 rows, 2652 columns and 2652 nonzeros
Model fingerprint: 0xf218a4b2
Variable types: 0 continuous, 2652 integer (2652 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.00s
Presolved: 52 rows, 2652 columns, 2652 nonzeros
Variable types: 0 continuous, 2652 integer (2652 binary)

Root relaxation: objective 7.281303e+03, 85 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7281.30305    0    -          - 7281.3

##### 3.4) Instância 4

In [19]:
# Instance range
inst1 = range(len(distance_matrices['Instance100']))
# print(inst1)

# Instance size
num_nodes = distance_matrices['Instance100'].shape[0]
node_ids1 = list(range(num_nodes))
# print(nodesInst1)

# Criando o modelo
m1 = gp.Model()

# Criando variávies: nesse caso todos o nós são adjacente entre si
variables = m1.addVars(pairwise_distances4.keys(),
                     obj=pairwise_distances4,
                     vtype=gp.GRB.BINARY,
                     name='x')

# Direções simétricas: copiando objeto
for u, v in pairwise_distances4.keys():
    variables[v, u] = variables[u, v] # mesmo arco porém na direção oposta

# Criando restrições
# Em cada nó pode haver somente dois arcos selecionados - um nó de chegada e outro nó de saída
cons = m1.addConstrs(variables.sum(node,'*') == 2 for node in inst1)

#### Registrando tempo de início
start_time = time.time()

# Adicionando restrições de sub-rotas de Dantzig de forma iterativa
# As restrições serão inseridas quando sub-rotas forem criadas
def subtourelim(model, where):
    # Importando biblioteca auxiliar
    from itertools import combinations

    if where == gp.GRB.Callback.MIPSOL:
        # Criando uma lista de arcos selecionados
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)

        # Encontrando a menor subrota criada dentro da atual lista de arcos selecionados
        tour = subtour(selected)
        if len(tour) < len(node_ids1):
            # Adiciona restrição de eliminação de sub-rotas para o par de cidade na subrota
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Dada uma sublista de arcos encontre a menor subrota
def subtour(edges):
    unvisited = node_ids1[:]
    cycle = node_ids1[:] # Proxy - será substituída
    while unvisited:  # 'True' se a lista não estiver vazia
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nova menor subrota
    return cycle

# Resolvendo o modelo
m1._vars = variables
m1.Params.lazyConstraints = 1
m1.optimize(subtourelim)

#### Registrando tempo de término
end_time = time.time()

# Solução
vals = m1.getAttr('x', variables)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

# Rota a ser traçada
rota1 = subtour(selected)

#### Calculando tempo decorrido
elapsed_time_DT1 = round(end_time - start_time, 4)

print('________________________________________________________________')
#### Retornando solução
print('\nInstância 4 - Função Objetivo: R$', round(m1.objVal, 2))

#### Retornando tempo decorrido
print("\nElapsed time:", elapsed_time_DT1, "seconds")
print('\nRota:', rota1)
print('________________________________________________________________')


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 100 rows, 9900 columns and 9900 nonzeros
Model fingerprint: 0xc1ce0230
Variable types: 0 continuous, 9900 integer (9900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.02s
Presolved: 100 rows, 9900 columns, 9900 nonzeros
Variable types: 0 continuous, 9900 integer (9900 binary)

Root relaxation: objective 1.994385e+04, 159 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 19943.8534    0   18          - 199

##### 3.5) Instância 5

In [20]:
# Instance range
inst1 = range(len(distance_matrices['Instance130']))
# print(inst1)

# Instance size
num_nodes = distance_matrices['Instance130'].shape[0]
node_ids1 = list(range(num_nodes))
# print(nodesInst1)

# Criando o modelo
m1 = gp.Model()

# Criando variávies: nesse caso todos o nós são adjacente entre si
variables = m1.addVars(pairwise_distances5.keys(),
                     obj=pairwise_distances5,
                     vtype=gp.GRB.BINARY,
                     name='x')

# Direções simétricas: copiando objeto
for u, v in pairwise_distances5.keys():
    variables[v, u] = variables[u, v] # mesmo arco porém na direção oposta

# Criando restrições
# Em cada nó pode haver somente dois arcos selecionados - um nó de chegada e outro nó de saída
cons = m1.addConstrs(variables.sum(node,'*') == 2 for node in inst1)

#### Registrando tempo de início
start_time = time.time()

# Adicionando restrições de sub-rotas de Dantzig de forma iterativa
# As restrições serão inseridas quando sub-rotas forem criadas
def subtourelim(model, where):
    # Importando biblioteca auxiliar
    from itertools import combinations

    if where == gp.GRB.Callback.MIPSOL:
        # Criando uma lista de arcos selecionados
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)

        # Encontrando a menor subrota criada dentro da atual lista de arcos selecionados
        tour = subtour(selected)
        if len(tour) < len(node_ids1):
            # Adiciona restrição de eliminação de sub-rotas para o par de cidade na subrota
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Dada uma sublista de arcos encontre a menor subrota
def subtour(edges):
    unvisited = node_ids1[:]
    cycle = node_ids1[:] # Proxy - será substituída
    while unvisited:  # 'True' se a lista não estiver vazia
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nova menor subrota
    return cycle

# Resolvendo o modelo
m1._vars = variables
m1.Params.lazyConstraints = 1
m1.optimize(subtourelim)

#### Registrando tempo de término
end_time = time.time()

# Solução
vals = m1.getAttr('x', variables)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

# Rota a ser traçada
rota1 = subtour(selected)

#### Calculando tempo decorrido
elapsed_time_DT1 = round(end_time - start_time, 4)

print('________________________________________________________________')
#### Retornando solução
print('\nInstância 5 - Função Objetivo: R$', round(m1.objVal, 2))

#### Retornando tempo decorrido
print("\nElapsed time:", elapsed_time_DT1, "seconds")
print('\nRota:', rota1)
print('________________________________________________________________')


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 130 rows, 16770 columns and 16770 nonzeros
Model fingerprint: 0xb8fad695
Variable types: 0 continuous, 16770 integer (16770 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.03s
Presolved: 130 rows, 16770 columns, 16770 nonzeros
Variable types: 0 continuous, 16770 integer (16770 binary)

Root relaxation: objective 7.429119e+03, 203 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7429.11915    0   26       

#### 4) Solving Approach: Matheuristics

#### 5) Results and Discussion

#### 6) Conclusion

### End